# Description


-   Get initial data from relation database and export them to csv for easier manipulation.


# Start


In [ ]:
import json
import os
from pathlib import Path

import pandas as pd
import psycopg2
from sqlalchemy import create_engine

from src.utils import load_data_config

In [ ]:
data_paths, column_types = load_data_config()
data_path_initial = Path(data_paths["initial"])

In [ ]:
def create_engine_from_config():
    with open("../../config/config.json", "r") as file:
        config = json.load(file)

    running_in_docker = os.environ.get("RUNNING_IN_DOCKER", "false").lower() == "true"

    db_config = config["relational_database_docker"] if running_in_docker else config["relational_database"]

    rel_host = db_config["host"]
    rel_port = db_config["port"]
    rel_db_name = db_config["dbname"]
    rel_user = db_config["user"]
    rel_password = db_config["password"]

    print(f"Connecting to database at {rel_host}:{rel_port} as {rel_user}")
    print(f"Running in Docker: {running_in_docker}")

    return create_engine(f"postgresql+psycopg2://{rel_user}:{rel_password}@{rel_host}:{rel_port}/{rel_db_name}")

In [ ]:
engine = create_engine_from_config()

# Data Processing


## Export Tables to CSV


In [ ]:
def export_informative_tables_to_cvs() -> None:
    """
    Reads informative tables from the relational database and exports them to csv files.
    """
    informative_tables = ["cats", "cat_informations", "cat_references", "breeds"]

    os.makedirs(data_path_initial, exist_ok=True)

    for table in informative_tables:
        df = pd.read_sql_query(f"SELECT * FROM {table}", engine)
        for column, dtype in column_types.items():
            if column in df.columns and dtype == "Int64":
                df[column] = df[column].astype("Int64")

        df.to_csv(f"{data_path_initial}/{table}.csv", index=False)


export_informative_tables_to_cvs()

## Combine Tables


In [ ]:
def combine_tables_and_export() -> None:
    """
    Merge the informative tables into one table and export it as all_cats.csv.
    """
    cats_df = pd.read_csv(f"{data_path_initial}/cats.csv", dtype=column_types, low_memory=False)
    cat_informations_df = pd.read_csv(
        f"{data_path_initial}/cat_informations.csv",
        dtype=column_types,
        low_memory=False,
    )
    cat_references_df = pd.read_csv(
        f"{data_path_initial}/cat_references.csv", dtype=column_types, low_memory=False
    )
    breeds_df = pd.read_csv(f"{data_path_initial}/breeds.csv", dtype=column_types, low_memory=False)

    cats_df = cats_df.drop(columns=["created_at", "updated_at", "deleted_at", "src_id"])
    cat_informations_df = cat_informations_df.drop(columns=["id"])
    cat_references_df = cat_references_df.drop(columns=["id"])

    df = pd.merge(cats_df, cat_informations_df, how="left", left_on="id", right_on="cat_id")
    df = df.drop(columns=["cat_id"])
    df = pd.merge(df, cat_references_df, how="left", left_on="id", right_on="cat_id")
    df = df.drop(columns=["cat_id"])
    df = pd.merge(df, breeds_df, how="left", left_on="breed_id", right_on="id")
    df = df.drop(columns=["breed_id", "id_y"])
    df = df.rename(columns={"id_x": "id", "code": "breed_code"})

    df = df.sort_values(by="id")

    df.to_csv(f"{data_path_initial}/all_cats.csv", index=False)


combine_tables_and_export()